In [9]:
import numpy as np
import pandas as pd

def read_fsgd(fsgd_path):
    """Read FSGD file and return structured data"""
    subjects = []
    groups = []
    tcho = []
    ages = []
    sexes = []
    with open(fsgd_path, 'r') as f:
        for line in f:
            if line.startswith('Input'):
                parts = line.strip().split()
                subjects.append(parts[1])
                groups.append(parts[2])
                tcho.append(parts[3])
                ages.append(float(parts[4]))
                sexes.append(int(parts[5]))
    return pd.DataFrame({
        'subject': subjects,
        'group': groups,
        'tcho' : tcho,
        'age': ages,
        'sex': sexes
    })

def create_design_matrix(df):
    """Create DODS design matrix"""
    n_subjects = len(df)
    design_matrix = np.zeros((n_subjects, 6))
    for i, row in df.iterrows():
        
        design_matrix[i, 4] = row['age']  
        design_matrix[i, 5] = row['sex']
        
        if row['group'] == 'HC':
            design_matrix[i, 0] = 1  # HC intercept
            design_matrix[i, 2] = row['tcho']
        else:  # EP group
            design_matrix[i, 1] = 1  # EP intercept
            design_matrix[i, 3] = row['tcho']
            
    return design_matrix

def save_design_matrix_for_glmfit(matrix, output_path):
    """Save design matrix in format required by mri_glmfit"""
    # Save as space-separated values with no header
    np.savetxt(output_path, 
               matrix,
               fmt='%.6f',     # 6 decimal precision
               delimiter=' ',   # space-separated
               comments='',     # no comments
               header='')      # no header

# Main execution
if __name__ == "__main__":
    fsgd_path = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/glm/tcho_analysis.fsgd"
    output_path = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/glm/tcho_design.mat"  # Changed to .mat extension
    
    # Read FSGD and create design matrix
    df = read_fsgd(fsgd_path)
    design_matrix = create_design_matrix(df)
    
    # Save in GLMfit format
    save_design_matrix_for_glmfit(design_matrix, output_path)
    
    # Print verification info
    print("\nDesign matrix shape:", design_matrix.shape)
    print("\nFirst 5 rows of design matrix:")
    print(design_matrix[:5, :])


Design matrix shape: (64, 6)

First 5 rows of design matrix:
[[ 0.        1.        0.        3.594643 35.5       0.      ]
 [ 0.        1.        0.        2.252458 35.666667  0.      ]
 [ 0.        1.        0.        3.160617 26.833333  0.      ]
 [ 0.        1.        0.        2.378054 18.416667  0.      ]
 [ 0.        1.        0.        2.766287 31.333333  1.      ]]
